In [1]:
!pip install optuna catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import optuna
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/Euron')

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [6]:
#train.drop(columns=['ID', 'Book-ID', 'Year-Of-Publication'], inplace=True)
#test.drop(columns=['ID', 'Book-ID', 'Year-Of-Publication'], inplace=True)
# location 전처리
# Location 분리
train['Location'] = train['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')
train['city'] = train['Location'].apply(lambda x: x.split(',')[0].strip())
train['state'] = train['Location'].apply(lambda x: x.split(',')[1].strip())
train['country'] = train['Location'].apply(lambda x: x.split(',')[2].strip())

train.drop('Location', axis=1, inplace=True)
train = train.replace('na', np.nan)
train = train.replace('', np.nan)

# Location 최빈값으로 대체
mode_city = train['city'].mode()[0]
mode_state = train['state'].mode()[0]
mode_country = train['country'].mode()[0]

# 결측값 대체
train['city'].fillna(mode_city, inplace=True)
train['state'].fillna(mode_state, inplace=True)
train['country'].fillna(mode_country, inplace=True)

<ipython-input-6-4af6f511e24b>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Location'] = train['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')


In [7]:
## Age 이상치 처리
# 이상치: 5세 이하 90세 초과
train.loc[(train['Age'] <= 5)|(train['Age'] > 90), 'Age'] = np.nan
train['Age'].isnull().sum() # 5세 이하 또는 90세 초과인 데이터 4497개
# 평균값으로 대체
train['Age'].mean() # Age (이상치 제거 후) 평균: 36.541634752034845
train['Age'] = train['Age'].fillna(train['Age'].mean())

## Age 범주화
# Age_group: 이상치 처리 & 5세 단위로 범주화
train['Age_group'] = pd.cut(train['Age'], bins = list(range(5, 91, 5)))
# Age_group2: 이상치 처리 & 미국 노동통계국 자료를 참고한 범주화
# train['Age_group'] = pd.cut(train['Age'], bins = [5, 6, 8, 12, 18, 25, 34, 44, 54, 64, 74, 91])

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder() #labelencoder함수를 가져온다.
encoder.fit(train['Age_group']) #내가 가지고 있는 데이터에 학습시킨다
digit_label = encoder.transform(train['Age_group'])
train['Age_group'] = digit_label # encoding 잘됐나?
#train['Age_group']

In [8]:
# 소문자 통일, 특수문자 제거 (종류: 15505 -> 15021)
train['Publisher'] = train['Publisher'].str.lower().replace('[^0-9a-zA-Z\s]', '', regex=True)

# 소문자 통일, 특수문자 제거, 띄어쓰기 제거 (종류: 15505 -> 14840)
train['Publisher'] = train['Publisher'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [9]:
train['Book-Author'] = train['Book-Author'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [10]:
# 출판년도 그룹화
# 만약 출판연도가 null이 있다면 정보없음(-1)로 채움
# train['Year-Of-Publication'] = train['Year-Of-Publication'].fillna(-1)

labels = ['Unknown', '-1800', '1800-1850', '1850-1900', '1900-1910', '1910-1920', '1920-1930', '1930-1940', '1940-1950','1950-1960', '1960-1970', '1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010-2020', '2020-']
bins = [-1, 0, 1800, 1850, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 3000]
train['Pub_gb'] = pd.cut(train['Year-Of-Publication'], bins, labels = labels,include_lowest = True)
train = train.drop(columns =['Year-Of-Publication'])
#train['Year-Of-Publication'] = pd.cut(train['Year-Of-Publication'], bins=bins, labels=labels)


encoder2 = LabelEncoder() #labelencoder함수를 가져온다.
encoder2.fit(train['Pub_gb']) #내가 가지고 있는 데이터에 학습시킨다
digit_label = encoder2.transform(train['Pub_gb'])
train['Pub_gb'] = digit_label # encoding 잘됐나?
#train['Pub_gb']

In [11]:
# Title 데이터 정리
import re # 정규표현 모듈

# 정규표현식으로 대소문자 통일, 특수문자 제거
train['Book-Title'] = [re.sub(r'[^0-9a-zA-Z:,]',  ' ',str(i)) for i in train['Book-Title']]
train['Main_Title'] = [i.split('  ')[0] for i in train['Book-Title']]
train['Sub_Title'] = [''.join(i.split('  ')[1:]) for i in train['Book-Title']]
train['Sub_Title'] = np.where(train['Sub_Title'] == '', 'No_SUB', train['Sub_Title'])

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871393 entries, 0 to 871392
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           871393 non-null  object 
 1   User-ID      871393 non-null  object 
 2   Book-ID      871393 non-null  object 
 3   Book-Rating  871393 non-null  int64  
 4   Age          871393 non-null  float64
 5   Book-Title   871393 non-null  object 
 6   Book-Author  871393 non-null  object 
 7   Publisher    871393 non-null  object 
 8   city         871393 non-null  object 
 9   state        871393 non-null  object 
 10  country      871393 non-null  object 
 11  Age_group    871393 non-null  int64  
 12  Pub_gb       871393 non-null  int64  
 13  Main_Title   871393 non-null  object 
 14  Sub_Title    871393 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 99.7+ MB


In [13]:
y = train['Book-Rating']

X = train.drop(['Book-Rating','Book-ID','Book-Title','ID'], axis=1)

In [14]:
kf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

cat_feature = ["User-ID",  "Age_group","Pub_gb","Publisher","country","Book-Author","state","city","Main_Title","Sub_Title"]

In [15]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

    params = {
        'objective' :  'RMSE',
        'n_estimators':trial.suggest_int("n_estimators", 30, 10000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'random_state' : 42,
        'task_type' : "GPU",
        'verbose' : False
    }


    model = CatBoostRegressor(
        **params,
        cat_features=["User-ID",  "Age_group","Pub_gb","Publisher","country","Book-Author","state","city","Main_Title","Sub_Title"]
    )
    model.fit(x_train, y_train, eval_set = [(x_test, y_test)], early_stopping_rounds = 500, verbose = 2000)

    pred = model.predict(x_test)

    mae = mean_squared_error(y_test, pred, squared=False)


    return mae

In [16]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=10)

[I 2024-02-08 04:17:01,698] A new study created in memory with name: no-name-2f34158e-fd9d-4e08-943b-595cfe4303aa
<ipython-input-15-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-15-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-15-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'random_strength': trial.suggest_uniform('random_strength',10,50),


0:	learn: 3.7327287	test: 3.7261244	best: 3.7261244 (0)	total: 417ms	remaining: 15m 48s
bestTest = 3.263974388
bestIteration = 82
Shrink model to first 83 iterations.


[I 2024-02-08 04:21:32,986] Trial 0 finished with value: 3.2639743968932007 and parameters: {'n_estimators': 2277, 'od_wait': 2152, 'learning_rate': 0.1646443965814705, 'reg_lambda': 0.5694107471452711, 'random_strength': 16.011565490103315, 'depth': 14, 'min_data_in_leaf': 10}. Best is trial 0 with value: 3.2639743968932007.
<ipython-input-15-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-15-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-15-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0

0:	learn: 3.6720106	test: 3.6611461	best: 3.6611461 (0)	total: 598ms	remaining: 1h 28m 52s
bestTest = 3.261350147
bestIteration = 122
Shrink model to first 123 iterations.


[I 2024-02-08 04:24:52,088] Trial 1 finished with value: 3.2613502353226975 and parameters: {'n_estimators': 8922, 'od_wait': 972, 'learning_rate': 0.3135561212272636, 'reg_lambda': 94.19760520316446, 'random_strength': 39.467861165075355, 'depth': 13, 'min_data_in_leaf': 14}. Best is trial 1 with value: 3.2613502353226975.
<ipython-input-15-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-15-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-15-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. 

...

[I 2024-02-08 04:42:39,883] Trial 7 finished with value: 3.2805317662313964 and parameters: {'n_estimators': 3934, 'od_wait': 897, 'learning_rate': 0.3021114288301562, 'reg_lambda': 1.108309939341069, 'random_strength': 12.44840802888191, 'depth': 15, 'min_data_in_leaf': 11}. Best is trial 2 with value: 3.252429009793645.
<ipython-input-15-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-15-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-15-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. Th

0:	learn: 3.5059358	test: 3.4749982	best: 3.4749982 (0)	total: 233ms	remaining: 26m 50s
bestTest = 3.292761272
bestIteration = 22
Shrink model to first 23 iterations.


[I 2024-02-08 04:44:34,934] Trial 8 finished with value: 3.2927611273052912 and parameters: {'n_estimators': 6927, 'od_wait': 2282, 'learning_rate': 0.8337333675326404, 'reg_lambda': 28.359423221445446, 'random_strength': 35.5415452680095, 'depth': 12, 'min_data_in_leaf': 10}. Best is trial 2 with value: 3.252429009793645.
<ipython-input-15-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-15-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-15-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. T

0:	learn: 3.5673084	test: 3.5429224	best: 3.5429224 (0)	total: 56.2ms	remaining: 3m 32s
2000:	learn: 3.2973728	test: 3.2595903	best: 3.2592907 (1781)	total: 1m 42s	remaining: 1m 31s
bestTest = 3.259164618
bestIteration = 2502
Shrink model to first 2503 iterations.


[I 2024-02-08 04:47:24,706] Trial 9 finished with value: 3.2591646640410357 and parameters: {'n_estimators': 3786, 'od_wait': 1158, 'learning_rate': 0.6699988159060445, 'reg_lambda': 63.286206389278114, 'random_strength': 44.22922035957264, 'depth': 3, 'min_data_in_leaf': 9}. Best is trial 2 with value: 3.252429009793645.


In [17]:
params=study.best_params
params['objective'] = 'RMSE'
params['random_state'] = 42
params['verbose'] = False
params['task_type'] = "GPU"
params['cat_features'] = ["User-ID",  "Age_group","Pub_gb","Publisher","country","Book-Author","state","city","Main_Title","Sub_Title"]

In [18]:
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

Best Score: 3.252429009793645
Best trial: {'n_estimators': 4822, 'od_wait': 1223, 'learning_rate': 0.19963138991116713, 'reg_lambda': 41.05265423218033, 'random_strength': 33.4850484271053, 'depth': 6, 'min_data_in_leaf': 12}


In [19]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [20]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

----test----

In [21]:
# Title 데이터 정리
import re # 정규표현 모듈

# 소문자로 통일, 특수문자 제거
test['Book-Title'] = test['Book-Title']\
                        .str.lower().replace('[^0-9a-zA-Z]', '', regex=True)
test['Main_Title'] = [i.split('  ')[0] for i in test['Book-Title']]
# SUB_title로 나누기
test['Sub_Title'] = [''.join(i.split('  ')[1:]) for i in test['Book-Title']]
# SUB_title이 없으면 No_SUB
test['Sub_Title'] = np.where(test['Sub_Title'] == '', 'No_SUB', test['Sub_Title'])

In [22]:
# 만약 출판연도가 null이 있다면 정보없음(-1)로 채움
test['Year-Of-Publication'] = test['Year-Of-Publication'].fillna(-1)
labels = ['Unknown', '-1800', '1800-1850', '1850-1900', '1900-1910', '1910-1920', '1920-1930', '1930-1940', '1940-1950','1950-1960', '1960-1970', '1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010-2020', '2020-']
bins = [-1, 0, 1800, 1850, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 3000]
test['Pub_gb'] = pd.cut(test['Year-Of-Publication'], bins, labels = labels,include_lowest = True)
test = test.drop(columns =['Year-Of-Publication'])
#train['Year-Of-Publication'] = pd.cut(train['Year-Of-Publication'], bins=bins, labels=labels)


encoder2 = LabelEncoder() #labelencoder함수를 가져온다.
encoder2.fit(test['Pub_gb']) #내가 가지고 있는 데이터에 학습시킨다
digit_label = encoder2.transform(test['Pub_gb'])
test['Pub_gb'] = digit_label

In [24]:
## Age 이상치 처리
# 이상치: 5세 이하 90세 초과
test.loc[(test['Age'] <= 5)|(test['Age'] > 90), 'Age'] = np.nan
test['Age'].isnull().sum() # 5세 이하 또는 90세 초과인 데이터 4497개
# 평균값으로 대체
test['Age'].mean() # Age (이상치 제거 후) 평균: 36.541634752034845
test['Age'] = test['Age'].fillna(test['Age'].mean())

## Age 범주화
# Age_group: 이상치 처리 & 5세 단위로 범주화
test['Age_group'] = pd.cut(test['Age'], bins = list(range(5, 91, 5)))
# 시각화로 분포 살펴보기
#train['Age_group'].value_counts().sort_index().plot.bar()
#plt.title('Age-group Distribution (Train)')
#plt.show()

# Age_group2: 이상치 처리 & 미국 노동통계국 자료를 참고한 범주화
# test['Age_group2'] = pd.cut(test['Age'], bins = [5, 6, 8, 12, 18, 25, 34, 44, 54, 64, 74, 91])
# 시각화로 분포 살펴보기
#train['Age_group2'].value_counts().sort_index().plot.bar()
#plt.title('Age-group2 Distribution (Train)')
#plt.show()

encoder = LabelEncoder() #labelencoder함수를 가져온다.
encoder.fit(test['Age_group']) #내가 가지고 있는 데이터에 학습시킨다
digit_label = encoder.transform(test['Age_group'])
test['Age_group'] = digit_label

In [25]:
# 소문자 통일, 특수문자 제거 (종류: 15505 -> 15021)
test['Publisher'] = test['Publisher'].str.lower().replace('[^0-9a-zA-Z\s]', '', regex=True)

# 소문자 통일, 특수문자 제거, 띄어쓰기 제거 (종류: 15505 -> 14840)
test['Publisher'] = test['Publisher'].str.lower().replace('[^0-9a-zA-Z\s]', '', regex=True)

In [26]:
# Location 분리
test['Location'] = test['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')
test['city'] = test['Location'].apply(lambda x: x.split(',')[0].strip())
test['state'] = test['Location'].apply(lambda x: x.split(',')[1].strip())
test['country'] = test['Location'].apply(lambda x: x.split(',')[2].strip())

test.drop('Location', axis=1, inplace=True)
test = test.replace('na', np.nan)
test = test.replace('', np.nan)

# Location 최빈값으로 대체
mode_city = test['city'].mode()[0]
mode_state = test['state'].mode()[0]
mode_country = test['country'].mode()[0]

# 결측값 대체
test['city'].fillna(mode_city, inplace=True)
test['state'].fillna(mode_state, inplace=True)
test['country'].fillna(mode_country, inplace=True)

print('각 칼럼별 최빈값')
print('국가:', mode_country, '\n주:', mode_state, '\n도시:', mode_city)
# train.isnull().sum()

<ipython-input-26-8141752042b3>:2: FutureWarning:

The default value of regex will change from True to False in a future version.



각 칼럼별 최빈값
국가: usa 
주: california 
도시: toronto


In [27]:
test['Book-Author'] = test['Book-Author'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [31]:
from sklearn.model_selection import StratifiedKFold
fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in fold.split(X, y):
        folds.append((train_idx, valid_idx))

test=test.drop(['Book-ID','Book-Title','ID'], axis=1)

In [32]:
from catboost import Pool

cat_pred_test = np.zeros((test.shape[0], 1))
for fold in range(10):
  print(f'/n----------------- Fold {fold} -----------------/n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_feature) # catboost에서 사용하는 데이터 형식인 Pool로 변환해주는것임.
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_feature)

  model_cat = CatBoostRegressor(**params)
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=50, verbose=100)

  cat_pred_test += model_cat.predict(test).reshape(-1, 1) / 10
  model_cat.save_model(f'CatBoostRegressor_{fold}.cbm')

/n----------------- Fold 0 -----------------/n
0:	learn: 3.7198401	test: 3.7117518	best: 3.7117518 (0)	total: 155ms	remaining: 12m 28s
100:	learn: 3.3278132	test: 3.2705813	best: 3.2705813 (100)	total: 8.86s	remaining: 6m 54s
200:	learn: 3.3151508	test: 3.2622388	best: 3.2622388 (200)	total: 18.2s	remaining: 6m 57s
300:	learn: 3.3093383	test: 3.2596517	best: 3.2596455 (299)	total: 27.7s	remaining: 6m 55s
400:	learn: 3.3046672	test: 3.2575512	best: 3.2575512 (400)	total: 36.2s	remaining: 6m 39s
500:	learn: 3.3004379	test: 3.2563551	best: 3.2563551 (500)	total: 45.6s	remaining: 6m 33s
600:	learn: 3.2966312	test: 3.2556519	best: 3.2556384 (598)	total: 55.1s	remaining: 6m 27s
700:	learn: 3.2931839	test: 3.2548550	best: 3.2548500 (699)	total: 1m 4s	remaining: 6m 19s
800:	learn: 3.2901074	test: 3.2544351	best: 3.2543957 (793)	total: 1m 12s	remaining: 6m 5s
900:	learn: 3.2869304	test: 3.2539587	best: 3.2539587 (900)	total: 1m 21s	remaining: 5m 56s
bestTest = 3.253958721
bestIteration = 900
Sh

In [36]:
submission['Book-Rating'] = cat_pred_test
# 평점은 0~10까지 존재하므로 그에 맞춰서 후처리
submission.loc[submission['Book-Rating'] > 10, 'Book-Rating'] = 10
submission.loc[submission['Book-Rating'] < 0, 'Book-Rating'] = 0
submission.describe()

,Book-Rating
count,159621.000000
mean,3.089393
std,1.999647
min,0.000000
25%,1.474573
50%,2.948548
75%,4.442420
max,10.000000


In [37]:
submission

,ID,Book-Rating
0,TEST_000000,5.341354
1,TEST_000001,6.146867
2,TEST_000002,6.535733
3,TEST_000003,6.110005
4,TEST_000004,6.495524
...,...,...
159616,TEST_159616,3.582729
159617,TEST_159617,3.652217
159618,TEST_159618,3.391038
159619,TEST_159619,3.497261
